In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [41]:
# Load the dataset
df = pd.read_csv('..\\..\\dataset\\Dataset_IoV.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']].values
y = df['Collussion'].values

# Define sequence length
num_steps = 3

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
X_scaled = X


In [42]:

# Function to create sequences
def create_sequences(X, y, num_steps):
    sequences, labels = [], []
    for i in range(len(X) - num_steps):
        sequences.append(X[i:i + num_steps])  # Take `num_steps` rows as input
        labels.append(y[i + num_steps])  # The next row is the target
    return np.array(sequences), np.array(labels)

# Create sequences
X_seq, y_seq = create_sequences(X_scaled, y, num_steps)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Check shapes
print(f"X_train shape: {X_train.shape}")  # (samples, 3, features)
print(f"X_test shape: {X_test.shape}")    # (samples, 3, features)

X_train shape: (165417, 3, 6)
X_test shape: (41355, 3, 6)


In [43]:
print(X_train.shape, y_train.shape,X_scaled.shape)
print(X_scaled.shape)
print(X_train[:2])

(165417, 3, 6) (165417,) (206775, 6)
(206775, 6)
[[[1.   2.   1.   0.35 0.   0.  ]
  [2.   2.   1.   0.35 1.   1.  ]
  [2.   2.   1.   0.35 0.   0.  ]]

 [[2.   1.   2.   1.   0.   0.  ]
  [2.   1.   1.   1.   0.   0.  ]
  [2.   1.   1.   1.   0.   0.  ]]]


# Model

In [44]:
print(y_train[:2])
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


[1 1]
X_train shape: (165417, 3, 6)
y_train shape: (165417,)
X_test shape: (41355, 3, 6)
y_test shape: (41355,)


In [45]:
def build_drnn_lstm():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(LSTM(units=100, activation='relu',return_sequences=True))
    model.add(LSTM(units=100, ))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [46]:
# Instantiate and train the model
model = build_drnn_lstm()
import time
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)  # Use test set for validation
)

end_time = time.time()

Epoch 1/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - loss: 0.6367 - mae: 0.4449 - mse: 0.2224 - val_loss: 0.6286 - val_mae: 0.4417 - val_mse: 0.2186
Epoch 2/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 0.6294 - mae: 0.4382 - mse: 0.2190 - val_loss: 0.6270 - val_mae: 0.4335 - val_mse: 0.2179
Epoch 3/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 0.6280 - mae: 0.4367 - mse: 0.2184 - val_loss: 0.6266 - val_mae: 0.4390 - val_mse: 0.2177
Epoch 4/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 0.6269 - mae: 0.4359 - mse: 0.2179 - val_loss: 0.6256 - val_mae: 0.4392 - val_mse: 0.2173
Epoch 5/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 0.6255 - mae: 0.4347 - mse: 0.2172 - val_loss: 0.6253 - val_mae: 0.4365 - val_mse: 0.2171
Epoch 6/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - loss: 0.6264 - mae: 0.4357 - mse: 0.2177 - val_loss: 0.6252 - val_mae: 0.4326 - val_mse: 0.2171
Epoch 7/10
1293/1293 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - loss: 0.6251 - mae

In [50]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 10 # e.g., 50
steps_per_epoch = 1293# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 12930
Time per Step: 0.016015 sec (16.01 ms)
Time per Step: 16014.83 µs
Time per Epoch: 20.71 sec


In [48]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


1293/1293 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
Mean Squared Error for y_pred (MSE): 0.2165349274873734
Root Mean Squared Error (RMSE): 0.4653331360298483
Accuracy: 0.6692298391971950


In [49]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()
